## Filtering using a cone-search (AKA spatial queries)
We've just inspected a Fermi event which gave us a position estimate and and error-circle. Next, let's see if there's any other recorded events which lie in the error circle:

In [ ]:
from __future__ import print_function
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
import voeventdb.remote as vr
import voeventdb.remote.apiv1 as apiv1
from voeventdb.remote.apiv1 import FilterKeys
from voeventdb.remote.helpers import Synopsis

In [ ]:
xrt_synopsis = Synopsis(apiv1.synopsis('ivo://nasa.gsfc.gcn/SWIFT#XRT_Pos_666352-553'))

In [ ]:
sky_event = xrt_synopsis.sky_events[0]

## Filtering using a cone-search (AKA spatial queries)

In [ ]:
cone = (sky_event.position, sky_event.position_error*3)
cone

In [ ]:
cone_filters = {
    FilterKeys.role: 'observation',
    FilterKeys.cone: cone
    }
apiv1.count(cone_filters)

In [ ]:
apiv1.stream_count(cone_filters)